In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd, scipy.signal as sig, matplotlib.pyplot as plt\n",
    "ssn = pd.read_csv(\"../data/ssn.csv\", parse_dates=[\"Date\"], index_col=\"Date\")\n",
    "\n",
    "# --- Grab global NDVI with Earth Engine (quick cell below) ---\n",
    "import ee\n",
    "ee.Initialize()\n",
    "ndvi = (ee.ImageCollection(\"MODIS/006/MOD13C2\")\n",
    "        .select('NDVI')\n",
    "        .filterDate('2000-01-01','2024-12-31')\n",
    "        .map(lambda img: img.set('date', img.date().format('YYYY-MM-01')))\n",
    "        .map(lambda img: img.multiply(0.0001)))  # scale factor\n",
    "ts = geemap.image_series_to_pandas(ndvi)\n",
    "ts.to_csv(\"../data/global_ndvi.csv\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# --- Lomb–Scargle power spectrum ---\n",
    "df = pd.read_csv(\"../data/global_ndvi.csv\", parse_dates=[\"date\"], index_col=\"date\").mean(axis=1)\n",
    "freq, power = sig.lombscargle(df.index.astype(np.int64)//1e9, df.values, \n",
    "                              1/np.linspace(1, 25, 5000))  # periods in years\n",
    "plt.semilogx(1/freq/365.25, power)\n",
    "plt.axvline(11, color='red')\n",
    "plt.title(\"NDVI 11-year power\")\n",
    "plt.savefig(\"../plots/ndvi_lomb.png\", dpi=300)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}